#### Install

```pip install ipywidgets```
```jupyter nbextension enable --py widgetsnbextension```

For the following command I would get an error which was fixed by installing Node.js which for Mac you can do by downloading it from their website https://nodejs.org/en/: 

```jupyter labextension install @jupyter-widgets/jupyterlab-manager```

#### Set parameters!

In [1]:
# imports
import ipywidgets as w
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
pd.set_option('display.max_colwidth', 0)

In [2]:
load = True            # if not the first time, and you want to load earlier saved data
index = 1      #'explanation'   # if loading annotated data, enter index where you left off last time
annotator_number = 1        # note which annotator number you have been given
# update for your own use
path = 'data/human_judgment.jsonl'

#### Functionality

In [3]:
# setting up data
if load == False:
    data = pd.read_json(path, lines=True)

    def htmlivize(val):
        return val.replace('|','<br><br>')
    data.dialogue = data.dialogue.transform(htmlivize)

    line = f" <br> -------------------------------------------------------------------------------------------------- <br>"
    summary_start = "<font color='blue'> Summary: </font> "
    data['text'] = data['dialogue'] + line + summary_start + data['summary']

    df = pd.DataFrame(list(data.text), columns = ['texts'])

    
    df['texts'] = df.texts.replace(replace_dict)
    df['coherence_results'] = 0
    df['consistency_results'] = 0
    df['relevance_results'] = 0
    df['fluency_results'] = 0
    df['comments'] = 'Any Comments?'

else:
    df = pd.read_csv(f'saved_df_ann{annotator_number}.csv', delimiter=';')


In [4]:
# explanation text
explanation = "This is the annotaiton tool. We will go through every dialogue where you will annotate each summary.\
                <br> 4 sliders are presented, adjust the sliders and click 'next'. the slider values are stored in the dataframe.\
                <br> The sliders are not reset when going to a new summary. You can go to the previous summary if you want to change something.\
                <br> Note that you will have to adjust the sliders again when you go to the next.\
                <br><br> When you are done, press 'finish'\
                <br> If you want to take a break, press pause. The data is then stored and you can continue next time."

In [5]:
# sliders
coherence = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Coherence:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

consistency = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Consistency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

relevance = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Relevance:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

fluency = w.IntSlider(value=3,
    min=1,
    max=5,
    step=1,
    description='Fluency:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

comments = w.Textarea(value='Any comments?')

In [6]:
# Prepare button functionality
def next(b):
    global df, index, out, progression
    
    if index == 'explanation':
        index = 0
        progress_text.value=f'Current index: {index}'
        
        comments.value=df.loc[index,'comments']
        coherence.value = df.loc[index,'coherence_results']
        consistency.value = df.loc[index,'consistency_results']
        relevance.value = df.loc[index,'relevance_results']
        fluency.value = df.loc[index,'fluency_results']
        text = w.HTMLMath(value=df.loc[index, 'texts'])
        
        with out:
            out.clear_output()
            display(text)

    elif index == len(df):
        progress_text.value=f'Current index: {index}'
        text = w.HTMLMath(value='STOP <br> You are done!!!')
        with out:
            out.clear_output()
            display(text)

    else:
        progression.value = index+1
        
        # add current slider values to df
        df.loc[index,'coherence_results'] = coherence.value
        df.loc[index,'consistency_results'] = consistency.value
        df.loc[index,'relevance_results'] = relevance.value
        df.loc[index,'fluency_results'] = fluency.value
        df.loc[index, 'comments'] = comments.value

        index+=1
        if index == len(df):
            text = w.HTMLMath(value='STOP <br> You are done!!!')
            progress_text.value=f'Current index: {index}'
            
            with out:
                out.clear_output()
                display(text)
        
        else:
            comments.value=df.loc[index,'comments']
            coherence.value = df.loc[index,'coherence_results']
            consistency.value = df.loc[index,'consistency_results']
            relevance.value = df.loc[index,'relevance_results']
            fluency.value = df.loc[index,'fluency_results']

            progress_text.value=f'Current index: {index}'
            text = w.HTMLMath(value=df.loc[index, 'texts'])
            # was trying to update the texts show, but does not work
            with out:
                out.clear_output()
                display(text)

def small_next(b):
    # update position
    global df, index, out, progression

    if index == 'explanation':
        index = 0
        progression.value = index+1

        comments.value=df.loc[index,'comments']
        coherence.value = df.loc[index,'coherence_results']
        consistency.value = df.loc[index,'consistency_results']
        relevance.value = df.loc[index,'relevance_results']
        fluency.value = df.loc[index,'fluency_results']

    else:
        index += 1
    
    if index == len(df):
        index=0

    progress_text.value=f'Current index: {index}'
    
    comments.value=df.loc[index,'comments']
    coherence.value = df.loc[index,'coherence_results']
    consistency.value = df.loc[index,'consistency_results']
    relevance.value = df.loc[index,'relevance_results']
    fluency.value = df.loc[index,'fluency_results']


    # refresh display
    text = w.HTMLMath(value=df.loc[index, 'texts'])
    with out:
        out.clear_output()
        display(text)

def back(b):
    # update position
    global df, index, out, progression

    if index == 'explanation':
        progression.value = 0
        index = 'explanation'
        progress_text.value=f'Current index: {index}'
        text = w.HTMLMath(value=explanation)

    else:
        index -= 1
        if index == -1:
            progression.value = 0
            index = 'explanation'
            progress_text.value=f'Current index: {index}'
            text = w.HTMLMath(value=explanation)

        

        else:
            progression.value = index+1
            progress_text.value=f'Current index: {index}'

            comments.value = df.loc[index,'comments']
            coherence.value = df.loc[index,'coherence_results']
            consistency.value = df.loc[index,'consistency_results']
            relevance.value = df.loc[index,'relevance_results']
            fluency.value = df.loc[index,'fluency_results']

            text = w.HTMLMath(value=df.loc[index, 'texts'])
        
        with out:
            out.clear_output()
            display(text)

def pause_annotation(b):
    global index, df
    # store temp data
    df.to_csv(f'saved_df_ann{annotator_number}.csv', sep=';',index=False)

    text = w.HTMLMath(value=f'You paused annotation! Data is stored on your local drive. <br>\
                            Next time, add below index in top of notebook to continue where you left off.\
                            <br><br> Stopped at index: {index}')
    with out:
        out.clear_output()
        display(text)


def finish(b):
    global df
    # store data
    df.to_csv(f'saved_df_ann{annotator_number}.csv', sep=';',index=False)


    # notify user
    text = w.HTMLMath(value=f'You have finished annotating! The data is stored on the drive')
    with out:
        out.clear_output()
        display(text)


In [7]:
# Compose widget

# set initial Dialogue
if index == 'explanation':
    text = w.HTMLMath(value=explanation)
    
else:
    text = w.HTMLMath(value=df.loc[index, 'texts'])

out = w.Output()
with out:
    out.clear_output()
    display(text)

# save work button
forward = w.Button(description='Save and Next', icon='forward')
forward.on_click(next)
small_forward = w.Button(description='Next', icon='arrow-right')
small_forward.on_click(small_next)

previous =  w.Button(description='Previous',icon='arrow-left')
previous.on_click(back)

pause = w.Button(description='Pause and store data',icon='download')
pause.on_click(pause_annotation)
fin = w.Button(description='Finish',icon='check')
fin.on_click(finish)

progression_bar = w.IntProgress(
    value=0,
    min=0,
    max=len(df),
    description='Progress',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'cyan'},
    orientation='horizontal'
)

progress_text = w.HTMLMath(value=f'Current index: {index}')


# sliders & buttons
sliders = w.VBox([relevance, consistency, fluency, coherence])
buttons = w.HBox([previous, small_forward, forward, pause, fin])
progression = w.HBox([progression_bar, progress_text])

#### Run!

In [8]:
# run widget
display(out)
display(sliders)
display(comments)
display(progression)
display(buttons)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': 'HTMLMath(value="<br><br> Elena: Happy …

Textarea(value='Any comments?')

In [9]:
# you can use this cell to check up on the current dataframe.
# But it is not needed, use the widget for storing data!
df.iloc[index-2:index+1]

,texts,coherence_results,consistency_results,relevance_results,fluency_results,comments
1104,"<br><br> Rita: I need some info on you client. <br><br> Helen: Which one? <br><br> Rita: Salvage Industries. <br><br> Helen: Yeah? What do you need? <br><br> Rita: What's their major business. <br><br> Helen: They are into waste disposal. <br><br> Rita: I thought so. <br><br> Rita: You think they are legit. <br><br> Helen: What do you mean?! <br><br> Helen: Of course, they are legit. <br><br> Rita: Got a favor to ask. Can you double check if they have no PR problems? <br><br> Helen: I don't think so. But I'll check it for you. <br><br> Rita: You are a dear, Helen. <br><br> Helen: I sure am:) <br> -------------------------------------------------------------------------------------------------- <br><font color='blue'> Summary: </font> Helen will double check if Rita's client Salvage Industries is legit.",4,3,4,5,Any comments?
1105,"<br><br> Rita: I need some info on you client. <br><br> Helen: Which one? <br><br> Rita: Salvage Industries. <br><br> Helen: Yeah? What do you need? <br><br> Rita: What's their major business. <br><br> Helen: They are into waste disposal. <br><br> Rita: I thought so. <br><br> Rita: You think they are legit. <br><br> Helen: What do you mean?! <br><br> Helen: Of course, they are legit. <br><br> Rita: Got a favor to ask. Can you double check if they have no PR problems? <br><br> Helen: I don't think so. But I'll check it for you. <br><br> Rita: You are a dear, Helen. <br><br> Helen: I sure am:) <br> -------------------------------------------------------------------------------------------------- <br><font color='blue'> Summary: </font> Rita needs some info on Helen's client Salvage Industries.",4,5,3,5,Any comments?
1106,"<br><br> Tricia: The cake is still not ready. <br><br> Zandra: Which cake? <br><br> Tricia: For your daughter’s birthday, Tam ;) <br><br> Zandra: Oh, of course, there are so many of them, I don’t even know what’s going on. <br><br> Tricia: Sure thing, you need a hand ;] <br><br> Zandra: Thank you so much, what would I do without you… <br><br> Zandra: But what about the cake, the party is tomorrow! <br><br> Tricia: You finally realized that… ;) <br><br> Tricia: They promised to have it tomorrow at Tricia: Zandra: . <br><br> Zandra: The party starts at Tricia: , it’s way too late! <br><br> Tricia: I’ll send Erwin to get it, he’ll deliver it to us by bike ;D <br><br> Zandra: Bike… God save us all… <br><br> Tricia: He’s a good driver and has a special bag, it will be ok, I promise :* <br><br> Zandra: Ok, how about the rest? I have all decorations, starting with them in the morning after kids go to grandma <br><br> Zandra: I know nothing about the food and outside attractions <br><br> Tricia: Food I’m taking care of, outside thing will be done by a company, they come and organize everything, don’t worry <br><br> Zandra: You’re the best, I have to make for it <3 <br><br> Tricia: Nooo, no way, it’s nothing. <br><br> Tricia: I know it’s not easy for you with 4 children. <br><br> Zandra: Noo, it’s not, but I regret nothing :* <br><br> Tricia: And that’s the most important thing! <br> -------------------------------------------------------------------------------------------------- <br><font color='blue'> Summary: </font> zandra's daughter , tam , has a birthday tomorrow . the party starts at tricia's place , so erwin will deliver a birthday cake by bike . zandra is taking care of the decorations , tricia of the food and a company of the outside attractions .",0,0,0,0,Any comments?
